In [1]:
import numpy as np
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup

In [2]:
cases = pd.read_csv("data100k.csv")

In [3]:
cases.head(3).T

,0,1,2
person_id,102090000000110,343221000000125,343221000000125
HearingDate,2019-02-28,2009-12-07,2011-01-20
CodeSection,A.46.2-862,B.46.2-301,A.46.2-707
codesection,covered elsewhere,covered elsewhere,covered elsewhere
ChargeType,Misdemeanor,Misdemeanor,Misdemeanor
chargetype,Misdemeanor,Misdemeanor,Misdemeanor
Class,1,1,3
DispositionCode,Guilty,Guilty,Guilty
disposition,Conviction,Conviction,Conviction
Plea,NaN,NaN,NaN


## What code sections are most frequent?

In [4]:
cases["CodeSection"].value_counts().reset_index().head(15)

,CodeSection,count
0,A.46.2-862,26379
1,B.46.2-301,25967
2,46.2-300,17934
3,C.46.2-862,11728
4,18.2-250.1,10573
5,A.18.2-266,8568
6,18.2-95,7561
7,18.2-250,6949
8,18.2-57,6699
9,A.46.2-852,6667


## Which ones most often lead to convictions?

In [5]:
cases["DispositionCode"].value_counts()
cases["conviction"] = [x == "Guilty" or x == "Guilty In Absentia" for x in cases["DispositionCode"]]
convict_rate = cases.groupby("CodeSection").agg({"conviction": ["mean", "count"]}).reset_index()
convict_rate.columns = ["CodeSection", "conviction_rate", "count"]
convict_rate = convict_rate.query("count > 30")
convict_rate.sort_values("conviction_rate", ascending=False)

,CodeSection,conviction_rate,count
1806,23-55,0.981818,55
1633,21-336,0.960000,50
1755,23-22.1(A),0.954198,131
2103,29-17(C),0.942857,70
4111,G.18.2-266,0.930233,43
...,...,...,...
1450,19.2-135,0.084337,83
2321,3.2-6503.1,0.071429,42
140,11.1-2,0.052632,38
1433,19.2-100,0.000000,238


In [6]:
cases.query("CodeSection == '23-55'")["fips"]

8585      810
17940     810
34148     810
34168     810
34169     810
46925     810
68396     810
120838    810
120844    810
120862    810
120864    810
121263    810
132073    810
153191    810
153206    810
153208    810
153210    810
153222    810
155161    810
157747    810
159548    810
163720    810
173408    810
192200    810
196647    810
200799    810
200828    810
200922    810
230213    810
240905    810
240909    810
240912    810
240914    810
240916    810
240917    810
240918    810
240919    810
240920    810
240922    810
240923    810
240926    810
240927    810
240945    810
240946    810
240948    810
249880    810
251381    810
257688    810
257697    810
257699    810
257739    810
257747    810
266159    810
291045    810
291048    810
Name: fips, dtype: int64

## Which ones have the most severe racial disparities?

In [7]:
cases["Race"].unique()

array(['Black(Non-Hispanic)', 'Hispanic', 'White Caucasian(Non-Hispanic)',
       'MISSING', 'Asian Or Pacific Islander', 'Black (Non-Hispanic)',
       'White Caucasian (Non-Hispanic)',
       'Other(Includes Not Applicable.. Unknown)',
       'Other (Includes Not Applicable.. Unknown)', 'Black', 'White',
       'Unknown (Includes Not Applicable.. Unknown)', 'American Indian',
       'Unknown', 'Asian or Pacific Islander',
       'American Indian Or Alaskan Native'], dtype=object)

In [8]:
replace_map = {'Black(Non-Hispanic)':"Black", 
               'Hispanic':"Hispanic", 
               'White Caucasian(Non-Hispanic)':"White",
               'MISSING':"Missing/Other", 
               'Asian Or Pacific Islander':"Asian or Pacific Islander", 
               'Black (Non-Hispanic)':"Black",
               'White Caucasian (Non-Hispanic)':"White",
               'Other(Includes Not Applicable.. Unknown)':"Missing/Other",
               'Other (Includes Not Applicable.. Unknown)':"Missing/Other", 
               'Black':"Black", 
               'White':"White",
               'Unknown (Includes Not Applicable.. Unknown)':"Missing/Other", 
               'American Indian':"American Indian or Alaskan Native",
               'Unknown':"Missing/Other", 
               'Asian or Pacific Islander':"Asian or Pacific Islander",
               'American Indian Or Alaskan Native':"American Indian or Alaskan Native"}
cases["Race"] = cases["Race"].replace(replace_map)
cases["Race"].value_counts()

Race
White                                159627
Black                                115627
Hispanic                               9319
Missing/Other                          5928
Asian or Pacific Islander              2794
American Indian or Alaskan Native       303
Name: count, dtype: int64

In [40]:
#I choose to analyze only the convictions
cases_convict = cases.query("conviction == True")
cases_convict_race = cases_convict.groupby(["CodeSection","Race", "fips"]).size().reset_index()
cases_convict_race = cases_convict_race.rename({0:"count"}, axis=1)
cases_convict_race

,CodeSection,Race,fips,count
0,01-2007,White,51,1
1,1,Black,550,3
2,1,White,550,1
3,1-12,Black,650,27
4,1-12,White,650,6
...,...,...,...,...
27500,Z.18.2-91,White,840,2
27501,Z.18.2-91; 26,Black,700,1
27502,Z.18.2-95,Black,67,1
27503,Z.18.2-95,Black,83,1


In [42]:
cases_reshape = cases_convict_race.pivot_table(index = ["CodeSection", "fips"],
                               columns = "Race", 
                               values = "count",
                                fill_value=0).reset_index()
cases_reshape

Race,CodeSection,fips,American Indian or Alaskan Native,Asian or Pacific Islander,Black,Hispanic,Missing/Other,White
0,01-2007,51,0.0,0.0,0.0,0.0,0.0,1.0
1,1,550,0.0,0.0,3.0,0.0,0.0,1.0
2,1-12,650,0.0,0.0,27.0,0.0,0.0,6.0
3,1-200,29,0.0,0.0,1.0,0.0,0.0,0.0
4,1-200,105,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
18700,Z.18.2-91,840,0.0,0.0,0.0,0.0,0.0,2.0
18701,Z.18.2-91; 26,700,0.0,0.0,1.0,0.0,0.0,0.0
18702,Z.18.2-95,67,0.0,0.0,1.0,0.0,0.0,0.0
18703,Z.18.2-95,83,0.0,0.0,1.0,0.0,0.0,0.0


In [43]:
cases_reshape = cases_reshape.assign(total = cases_reshape["American Indian or Alaskan Native"] +
                                    cases_reshape["Asian or Pacific Islander"] +
                                    cases_reshape["Black"] +
                                    cases_reshape["Hispanic"] +
                                    cases_reshape["Missing/Other"] +
                                    cases_reshape["White"])
cases_reshape = cases_reshape.query("total >= 50")
cases_reshape = cases_reshape.assign(black_pc = cases_reshape["Black"]/cases_reshape["total"],
                                     white_pc = cases_reshape["White"]/cases_reshape["total"])
cases_reshape.sort_values("black_pc", ascending=False)

Race,CodeSection,fips,American Indian or Alaskan Native,Asian or Pacific Islander,Black,Hispanic,Missing/Other,White,total,black_pc,white_pc
3040,18.2-248,760,0.0,0.0,228.0,0.0,0.0,3.0,231.0,0.987013,0.012987
1247,18.2-119,740,0.0,0.0,132.0,0.0,0.0,5.0,137.0,0.963504,0.036496
9738,24-253,740,0.0,0.0,110.0,0.0,0.0,5.0,115.0,0.956522,0.043478
2986,18.2-248,117,0.0,0.0,55.0,0.0,0.0,3.0,58.0,0.948276,0.051724
17297,B.46.2-301,760,0.0,0.0,51.0,0.0,0.0,3.0,54.0,0.944444,0.055556
...,...,...,...,...,...,...,...,...,...,...,...
1958,18.2-172,139,0.0,0.0,0.0,0.0,0.0,98.0,98.0,0.000000,1.000000
5607,18.2-374.1:1,169,0.0,0.0,0.0,0.0,0.0,137.0,137.0,0.000000,1.000000
1916,18.2-172,19,0.0,0.0,0.0,0.0,0.0,53.0,53.0,0.000000,1.000000
1947,18.2-172,105,0.0,0.0,0.0,0.0,0.0,94.0,94.0,0.000000,1.000000


In [11]:
cases.query("CodeSection == '24-253'")["fips"]

2319      740
2325      740
4399      740
4400      740
4401      740
         ... 
262328    740
262331    740
273650    740
278506    740
288281    740
Name: fips, Length: 131, dtype: int64

## In what localities (fips) are these disparities most severe?

In [12]:
race_url = "https://virginia.box.com/shared/static/i8i5onrkveks849pkky0gwgxlax8d8fe.xlsx"
hisp_url = "https://virginia.box.com/shared/static/fegrn0p0igzl95snji3ku6edwu0hy3dj.xlsx"

In [19]:
race_pop = pd.read_excel(race_url, skiprows = [0,1,2,3,5,6,7])
race_pop

/opt/miniconda3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: '2021 Population Estimates'!$5:$141.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


,FIPS,Jurisdiction,Total Population,American Indian,Unnamed: 4,Asian,Unnamed: 6,Black,Unnamed: 8,Pacific Islander,Unnamed: 10,Two or more races,Unnamed: 12,White,Unnamed: 14
0,1,Accomack County,33246,441,0.013265,370,0.011129,9859,0.296547,79,0.002376,609,0.018318,23125,0.695572
1,3,Albemarle County,113535,1069,0.009416,7925,0.069802,12581,0.110812,242,0.002132,3210,0.028273,95210,0.838596
2,5,Alleghany County,14986,126,0.008408,92,0.006139,906,0.060456,17,0.001134,283,0.018884,14136,0.943280
3,7,Amelia County,13268,177,0.013340,138,0.010401,2759,0.207944,15,0.001131,259,0.019521,10445,0.787232
4,9,Amherst County,31273,495,0.015828,339,0.010840,6475,0.207048,55,0.001759,828,0.026477,24796,0.792888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,800,Suffolk city,96194,1052,0.010936,3115,0.032382,43068,0.447720,232,0.002412,2975,0.030927,51977,0.540335
129,810,Virginia Beach city,457672,6241,0.013636,44491,0.097212,104827,0.229044,1846,0.004033,20837,0.045528,324018,0.707970
130,820,Waynesboro city,22550,309,0.013703,522,0.023149,3665,0.162528,35,0.001552,792,0.035122,18840,0.835477
131,830,Williamsburg city,15590,202,0.012957,1164,0.074663,2754,0.176652,37,0.002373,591,0.037909,12055,0.773252


In [21]:
race_pop.columns

Index(['FIPS', 'Jurisdiction', 'Total Population', 'American Indian',
       'Unnamed: 4', 'Asian ', 'Unnamed: 6', 'Black', 'Unnamed: 8',
       'Pacific Islander', 'Unnamed: 10', 'Two or more races', 'Unnamed: 12',
       'White ', 'Unnamed: 14'],
      dtype='object')

In [23]:
race_pop = race_pop[['FIPS', 'Jurisdiction', 'Total Population', 'American Indian', 
                     'Asian ', 'Black', 'Pacific Islander', 'Two or more races', 'White ']]

In [25]:
race_pop = race_pop.rename({'FIPS': 'fips', 
                            'Jurisdiction': 'jurisdiction', 
                            'Total Population': 'total_pop', 
                            'American Indian': 'amerind_pop', 
                            'Asian ': 'asian_pop', 
                            'Black': 'black_pop', 
                            'Pacific Islander': 'pacific_island_pop', 
                            'Two or more races': 'two_or_more_races_pop', 
                            'White ': 'white_pop'}, axis = 1)

In [26]:
race_pop

,fips,jurisdiction,total_pop,amerind_pop,asian_pop,black_pop,pacific_island_pop,two_or_more_races_pop,white_pop
0,1,Accomack County,33246,441,370,9859,79,609,23125
1,3,Albemarle County,113535,1069,7925,12581,242,3210,95210
2,5,Alleghany County,14986,126,92,906,17,283,14136
3,7,Amelia County,13268,177,138,2759,15,259,10445
4,9,Amherst County,31273,495,339,6475,55,828,24796
...,...,...,...,...,...,...,...,...,...
128,800,Suffolk city,96194,1052,3115,43068,232,2975,51977
129,810,Virginia Beach city,457672,6241,44491,104827,1846,20837,324018
130,820,Waynesboro city,22550,309,522,3665,35,792,18840
131,830,Williamsburg city,15590,202,1164,2754,37,591,12055


In [30]:
hisp_pop = pd.read_excel(hisp_url, skiprows = [0,1,2,3,5,6,7,8,9])
hisp_pop

/opt/miniconda3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: '2021 Population Estimates'!$5:$143.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


,FIPS,Jurisdiction,"Decennial Census Count, April 1, 2010",Unnamed: 3,Unnamed: 4,"Population Estimate, July 1, 2021",Unnamed: 6,Unnamed: 7,"April 1, 2010 - July 1, 2021",Unnamed: 9
0,1,"Accomack County, Virginia",33164,2850,8.593656,33246,3170,0.095350,320,0.112281
1,3,"Albemarle County, Virginia",98970,5417,5.473376,113535,6750,0.059453,1333,0.246077
2,5,"Alleghany County, Virginia",16250,176,1.083077,14986,265,0.017683,89,0.505682
3,7,"Amelia County, Virginia",12690,290,2.285264,13268,507,0.038212,217,0.748276
4,9,"Amherst County, Virginia",32353,625,1.931815,31273,849,0.027148,224,0.358400
...,...,...,...,...,...,...,...,...,...,...
128,800,"Suffolk city, Virginia",84585,2415,2.855116,96194,4684,0.048693,2269,0.939545
129,810,"Virginia Beach city, Virginia",437994,28987,6.618127,457672,40525,0.088546,11538,0.398041
130,820,"Waynesboro city, Virginia",21006,1337,6.364848,22550,2244,0.099512,907,0.678384
131,830,"Williamsburg city, Virginia",14068,941,6.688939,15590,1183,0.075882,242,0.257173


In [32]:
hisp_pop = hisp_pop[['FIPS', 'Unnamed: 6']]
hisp_pop = hisp_pop.rename({'FIPS': 'fips',
                            'Unnamed: 6': 'hisp_pop'}, axis=1)
hisp_pop

,fips,hisp_pop
0,1,3170
1,3,6750
2,5,265
3,7,507
4,9,849
...,...,...
128,800,4684
129,810,40525
130,820,2244
131,830,1183


# Things that can go wrong (without programming error) on merge

1. Rows that should match are unmatched, get missing data if full/outer join, get deleted in inner join
2. One-to-one merge actually many-to-one or many-to-many

Small datasets should be merged twice: with checks and without

In [36]:
pop = pd.merge(race_pop, hisp_pop, 
               on = 'fips', 
               how = 'outer',
               validate = 'one_to_one',
               indicator = 'matched')

In [37]:
pop['matched'].value_counts()

matched
both          133
left_only       0
right_only      0
Name: count, dtype: int64

In [38]:
pop = pd.merge(race_pop, hisp_pop, 
               on = 'fips', 
               how = 'inner')

In [33]:
data1 = [{'Country':'USA', 'Value1': 5},
         {'Country':'France', 'Value1': 15},
         {'Country':'France', 'Value1': 16},
         {'Country':'China', 'Value1': 50},
         {'Country':'Spain', 'Value1': 25},
         {'Country':'UK', 'Value1': .5},
         {'Country':'Thailand', 'Value1': 500}]
data1 = pd.DataFrame.from_records(data1)

data2 = [{'Country':'USA', 'Value2': 3},
         {'Country':'France', 'Value2': 13},
         {'Country':'France', 'Value2': 14},
         {'Country':'China', 'Value2': 30},
         {'Country':'Spain', 'Value2': 23},
         {'Country':'UK', 'Value2': .3},
         {'Country':'Thailand', 'Value2': 300}]
data2 = pd.DataFrame.from_records(data2)

In [34]:
pd.merge(data1, data2, on='Country')

,Country,Value1,Value2
0,USA,5.0,3.0
1,France,15.0,13.0
2,France,15.0,14.0
3,France,16.0,13.0
4,France,16.0,14.0
5,China,50.0,30.0
6,Spain,25.0,23.0
7,UK,0.5,0.3
8,Thailand,500.0,300.0


In [35]:
data1 = [{'Country':'USA', 'Value1': 5},
         {'Country':'France', 'Value1': 15},
         {'Country':'France', 'Value1': 16},
         {'Country':'China', 'Value1': 50},
         {'Country':'Spain', 'Value1': 25},
         {'Country':'UK', 'Value1': .5},
         {'Country':'Thailand', 'Value1': 500}]
data1 = pd.DataFrame.from_records(data1)

data2 = [{'Country':'United States', 'Value2': 3},
         {'Country':'France', 'Value2': 13},
         {'Country':'France', 'Value2': 14},
         {'Country':'China', 'Value2': 30},
         {'Country':'Spain', 'Value2': 23},
         {'Country':'UK', 'Value2': .3},
         {'Country':'Thailand', 'Value2': 300}]
data2 = pd.DataFrame.from_records(data2)
pd.merge(data1, data2, on='Country')

,Country,Value1,Value2
0,France,15.0,13.0
1,France,15.0,14.0
2,France,16.0,13.0
3,France,16.0,14.0
4,China,50.0,30.0
5,Spain,25.0,23.0
6,UK,0.5,0.3
7,Thailand,500.0,300.0


In [48]:
cases_pop = pd.merge(cases_reshape, pop,
                     on = 'fips',
                     how = 'outer',
                     validate = 'many_to_one',
                     indicator = 'matched')
cases_pop['matched'].value_counts()

matched
both          646
right_only     42
left_only       0
Name: count, dtype: int64

In [46]:
cases_pop.query('matched=="left_only"')['fips'].unique()

array([701, 702, 711, 712, 761, 762, 764])

In [47]:
replace_map = {701: 700, 
               702: 700, 
               711: 710, 
               712: 710, 
               761: 760, 
               762: 760, 
               764: 760}

cases_reshape['fips'] = cases_reshape['fips'].replace(replace_map)

In [49]:
cases_pop.query('matched=="right_only"')['fips'].unique()

array([  7,  11,  17,  29,  36,  37,  45,  49,  51,  63,  65,  71,  77,
        79,  91,  95,  97, 101, 103, 111, 113, 115, 119, 125, 133, 141,
       145, 157, 159, 181, 530, 580, 610, 620, 660, 678, 683, 685, 720,
       735, 750, 820])

Reasons why rows might fail to match:

1. Differences in coding/spelling: recode the values in one dataset so that they match
2. Differences in coverages: nothing to do but collect new data (if feasible)

In [50]:
cases_pop = pd.merge(cases_reshape, pop,
                     on = 'fips',
                     how = 'inner')
cases_pop

,CodeSection,fips,American Indian or Alaskan Native,Asian or Pacific Islander,Black,Hispanic,Missing/Other,White,total,black_pc,white_pc,jurisdiction,total_pop,amerind_pop,asian_pop,black_pop,pacific_island_pop,two_or_more_races_pop,white_pop,hisp_pop
0,13-1-5,510,0.0,0.0,37.0,0.0,1.0,54.0,92.0,0.402174,0.586957,Alexandria city,154706,2285,12722,37719,535,5434,107355,25586
1,13-1-5.1,510,0.0,0.0,31.0,0.0,0.0,32.0,63.0,0.492063,0.507937,Alexandria city,154706,2285,12722,37719,535,5434,107355,25586
2,13-23,87,0.0,1.0,22.0,0.0,1.0,34.0,58.0,0.379310,0.586207,Henrico County,333554,3460,36049,108373,586,9162,195229,20951
3,14-21,41,0.0,1.0,24.0,8.0,6.0,60.0,99.0,0.242424,0.606061,Chesterfield County,370688,5137,17612,100588,975,11182,258916,37765
4,14.2-81,13,0.0,0.0,71.0,4.0,0.0,16.0,91.0,0.780220,0.175824,Arlington County,232965,3689,31145,26879,760,8882,180541,36284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,C.46.2-862,710,0.0,0.0,76.0,5.0,5.0,63.0,149.0,0.510067,0.422819,Norfolk city,235089,4292,12673,103373,1219,9507,124482,21484
642,C.46.2-862,730,2.0,4.0,74.0,7.0,6.0,62.0,155.0,0.477419,0.400000,Petersburg city,33429,406,582,26434,113,905,6836,1788
643,C.46.2-862,760,1.0,0.0,149.0,11.0,14.0,70.0,245.0,0.608163,0.285714,Richmond city,226604,3052,7416,106660,677,5869,115477,17639
644,C.46.2-862,800,0.0,0.0,32.0,0.0,0.0,24.0,56.0,0.571429,0.428571,Suffolk city,96194,1052,3115,43068,232,2975,51977,4684


In [51]:
cases_pop['black_overrep_index'] = cases_pop['black_pc']/(cases_pop['black_pop']/cases_pop['total_pop'])

In [54]:
cases_pop.sort_values('black_overrep_index', ascending=False)[['CodeSection', 'jurisdiction', 
                                                               'Black', 'total', 'black_pop',
                                                               'total_pop', 'black_overrep_index']].head(20)

,CodeSection,jurisdiction,Black,total,black_pop,total_pop,black_overrep_index
294,46.2-300,Carroll County,35.0,171.0,530,29048,11.217919
460,A.46.2-862,Carroll County,28.0,138.0,530,29048,11.120372
607,C.46.2-862,Carroll County,95.0,519.0,530,29048,10.032210
319,46.2-300,Smyth County,27.0,99.0,969,29477,8.296369
281,46.2-1172,Hanover County,81.0,95.0,11853,111603,8.028030
102,18.2-250,Rockingham County,16.0,54.0,3186,84394,7.848597
317,46.2-300,Scott County,9.0,84.0,301,21419,7.624229
493,A.46.2-862,Washington County,28.0,183.0,1110,53635,7.393196
129,18.2-250.1,Rockingham County,17.0,62.0,3186,84394,7.263117
632,C.46.2-862,Washington County,10.0,69.0,1110,53635,7.002872
